# Test 5 (paper) - Experimento 3 (PT)

Entrenamiento de Mask R-CNN con dataset modificado para emular "Test 5"

### Hiperparametros
* **epoch = 300**
    * steps x epoch = 146 (lotes de imagenes)
    * batch = 2
* optimizador = SGD
* Funcion de perdida = SMOOTHL1LOSS
* Metrica de evaluacion = mAP (IoU >= 0.5)
* Mini-mask shape: 56x56
* RPN anchor scales: (32, 64, 128, 256, 512)
* Tasa de aprendizaje: 0.001
* imagenes = 292
    * entrenamiento 81% = 237
    * validacion 19% = 55
* etiquetas = 9140
* **resolucion = 1920 x 1080**
* etiquetas = bounding box formato VOC XML
* numero de clases = 1 (arandano)
* **data augmentation = false**


## Comprobar directorio principal

In [1]:
!pwd && ls

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport/model-training
build				   master-test_5-fase_2.ipynb
dist				   master-test_5.ipynb
mask_rcnn.egg-info		   master-test_5_fase_2.ipynb
master-test_11.ipynb		   master-test_6.ipynb
master-test_11_fase_2-Copy1.ipynb  master-test_6_fase_2.ipynb
master-test_11_fase_2.ipynb	   mrcnn
master-test_12.ipynb		   old
master-test_12_fase_2.ipynb


# Importar bibliotecas

In [2]:
# bibliotecas basicas
import os
from os import listdir
import sys
import json
import datetime

#sys.path.append("/tf/PT_JoseVeloso/Mask_RCNN-master/")

# bibliotecas avanzadas 
from xml.etree import ElementTree
import skimage.draw
import cv2
import imgaug

# bibliotecas mask rcnn 
from mrcnn.utils import Dataset
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.visualize import display_instances
from mrcnn.utils import extract_bboxes
from mrcnn.utils import compute_ap
from mrcnn.model import load_image_gt
from mrcnn.model import mold_image
from mrcnn import visualize

# biblioteca matplotlib 
import matplotlib.pyplot as plt

# bibliotecas numpy 
import numpy as np
from numpy import zeros
from numpy import asarray
from numpy import expand_dims
from numpy import mean

# bibliotecas keras
from tensorflow.keras.preprocessing.image import load_img   #keras.preprocessing.image tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.preprocessing.image import img_to_array

# ignorar alertas de elementos que seran descontinuados
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

%matplotlib inline
#plt.show()

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import datetime

# Fase 1 - entrenamiento con una sola clase y etiquetas de Bounding Box

En este entremamiento se utiliza un conjunto de datos simple con imágenes etiquetadas con cuadros delimitadores y una clase llamada 'Daño'. En la siguiente sección se encuentra el código para el entrenamiento del modelo. Se incluyen comentarios para describir mejor el flujo del programa.

In [3]:
class DatasetArandanos(Dataset):
    
    # la funcion load_dataset es usada para cargar el dataset de entrenamiento y test
    def load_dataset(self, dataset_dir, is_train=True):
        
        # se agrega una clase que se necesita para clasificar, en este caso arandano
        self.add_class("dataset", 1, "arandano")
        
        # se concatena dataset_dir con /images y /annots
        images_dir = dataset_dir + '/images/'
        annotations_dir = dataset_dir + '/annots/'
        
        # is_train sera Verdadero si se esta entrenando el modelo y Falso si se esta testeando el modelo
        for filename in listdir(images_dir):
            
            # extraer image id
            image_id = filename[:-4] # se usa para omitir los últimos 4 caracteres: '.jpg' (en class_id.jpg)
            
            # si is_train es Verdadero se omiten todas las imágenes con id mayor que e iguales a 
            # aproximadamente el 80% del conjunto de datos es para entrenamiento
            
            if is_train and int(image_id) >= 41590 :
                #print("image_id: ", image_id)
                continue
            
            # si is_train no es Verdadero se omiten todas las imágenes con id menores a 
            if not is_train and int(image_id) < 41590:
                continue
            
            # se declara la ruta de la imagen y la ruta de las etiquetas 
            img_path = images_dir + filename
            ann_path = annotations_dir + image_id + '.xml'
            
            # usando la función add_image se pasan image_id, image_path y ann_path para que la 
            # imagen actual se agregue al conjunto de datos para entrenamiento o prueba
            self.add_image('dataset', image_id=image_id, path=img_path, annotation=ann_path)

    # funcino usada para extraer bouding boxes desde archivos etiquetados 
    def extract_boxes(self, filename):

        # se puede ver en las imágenes que estan etiquetadas, como se extraen los valores de ancho, alto y bndbox
        
        # <annotation>
        # <size>

        #       <width>640</width>

        #       <height>360</height>

        #       <depth>3</depth>

        # </size>


        # <object>

        #          <name>damage</name>

        #          <pose>Unspecified</pose>

        #          <truncated>0</truncated>

        #          <difficult>0</difficult>


        #          <bndbox>

        #                 <xmin>315</xmin>

        #                 <ymin>160</ymin>

        #                 <xmax>381</xmax>

        #                 <ymax>199</ymax>

        #          </bndbox>

        # </object>
        # </annotation>
        
        # para analizar los archivos .xml
        tree = ElementTree.parse(filename)
        
        # para obtener la raíz del archivo xml
        root = tree.getroot()
        
        # se agregan todas las coordenadas x, y en boxes para todas las instancias de un objeto
        boxes = list()
        
        # se encuentran todos los atributos con el nombre bndbox que existan para cada ground truth en la imagen
        for box in root.findall('.//bndbox'):
            xmin = int(box.find('xmin').text)
            ymin = int(box.find('ymin').text)
            xmax = int(box.find('xmax').text)
            ymax = int(box.find('ymax').text)
            coors = [xmin, ymin, xmax, ymax]
            boxes.append(coors)
        
        # extraer ancho y alto de la imagen
        width = int(root.find('.//size/width').text)
        height = int(root.find('.//size/height').text)
        
        # retorna boxes-> list, width-> int y height-> int 
        return boxes, width, height
    
    # this function calls on the extract_boxes method and is used to load a mask for each instance in an image
    # returns a boolean mask with following dimensions width * height * instances

    # esta función llama al método extract_boxes y se usa para cargar una máscara para cada instancia en una imagen 
    # devuelve una máscara booleana con las siguientes dimensiones ancho * alto * instancias
    def load_mask(self, image_id):
        
        # info apunta al image_id actual 
        info = self.image_info[image_id]
        
        # se obtiene la ruta de anotación de image_id que es dataset_dir/annots/image_id.xml 
        path = info['annotation']
        
        # se llama al método extract_boxes (arriba) para obtener bndbox del archivo .xml
        boxes, w, h = self.extract_boxes(path)
        
        # se crea una cantidad de len(boxes) de mascaras de alto 'h' y ancho 'w'
        masks = zeros([h, w, len(boxes)], dtype='uint8')
        
        # se agrega (append) el class_id 1 para arandano en nuestro caso a la variable
        class_ids = list()
        
        # se recorren todos los boxes y generamos máscaras (máscara de bndbox) y class id para cada instancia
        # las máscaras tendrán forma rectangular ya que hemos usado bndboxes para etiquetas
        # por ejemplo: si 2.jpg tiene tres objetos, tendremos las siguientes máscaras y class_ids.

        # 000000000 000000000 000001110 
        # 000011100 011100000 000001110
        # 000011100 011100000 000001110
        # 000000000 011100000 000000000
        #    1         1          1    <- class_ids
        for i in range(len(boxes)):
            box = boxes[i]
            row_s, row_e = box[1], box[3]
            col_s, col_e = box[0], box[2]
            masks[row_s:row_e, col_s:col_e, i] = 1
            class_ids.append(self.class_names.index('arandano'))
        
        # retorna mascaras y class_ids como arreglo
        return masks, asarray(class_ids, dtype='int32')
    
    # esta funciones toma el image_id y retorna la ruta de la imagen 
    def image_reference(self, image_id):
        info = self.image_info[image_id]
        return info['path']



In [4]:
# clase de configuracion arandano, aqui se pueden cambiar valores de hyper parameters 
class ConfigArandanos(Config):

    # nombre de la configuracion 
    NAME = "arandano_cfg_test_5_2_"    
    
    # clase arandano + clase background 
    NUM_CLASSES = 1 + 1
    
    # pasos por epoch y confianza minima    # STEPS_PER_EPOCH = cantidad de lotes/batchs
    #STEPS_PER_EPOCH = 73   # por epoch se entrenaran 73 lotes de 4 imagenes, dataset = 292
    STEPS_PER_EPOCH = 146
    
    # tasa de aprendizaje y momentum
    LEARNING_RATE=0.001
    LEARNING_MOMENTUM = 0.8
    
    # penalización de regularización
    WEIGHT_DECAY = 0.0001
    
    # el tamaño de la imagen está controlado por este parámetro
    IMAGE_MIN_DIM = 512
    
    # pasos de validación
    VALIDATION_STEPS = 50
    
    # número de regiones de interés generadas por imagen
    Train_ROIs_Per_Image = 200
    
    # escala de anclas RPN y proporciones (ratios) para encontrar la ROI
    RPN_ANCHOR_SCALES = (32, 64, 128, 256, 512)    # Longitud del lado del ancla cuadrada, en píxeles 
    RPN_ANCHOR_RATIOS = [0.5, 1, 1.5]   # Proporciones de anclas por cada celda (ancho/alto). Un valor de 1 representa un ancla cuadrada y 0,5 es un ancla ancha 
    
    MINI_MASK_SHAPE = (56, 56)
    
    
config = ConfigArandanos()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     2
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 2
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1024
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  512
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1024 1024    3]
LEARNING_MOMENTUM              0.8
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

In [5]:
cd /tf/PT_JoseVeloso/Mask_RCNN-master_matterport/

/tf/PT_JoseVeloso/Mask_RCNN-master_matterport


In [6]:
pesos = 'arandano_cfg_test_5_20220907T0914/mask_rcnn_arandano_cfg_0300.h5'

conjunto_datos = 'customImages/test_5_2'

In [7]:
# cargar dataset de entrenamiento
train_set = DatasetArandanos()
train_set.load_dataset(conjunto_datos, is_train=True)
train_set.prepare()

# cargar dataset de test 
test_set = DatasetArandanos()
test_set.load_dataset(conjunto_datos, is_train=False)
test_set.prepare()

# preparar la configuración llamando a la clase de configuración definida por el usuario
config = ConfigArandanos()

# definir el modelo
with tf.device(config.DEVICE):
    model = MaskRCNN(mode='training', model_dir='./', config=config)

# cargar pesos del modelo mscoco
weights_path = pesos

# cargar los pesos del modelo
model.load_weights(weights_path, 
                   by_name=True, 
                   exclude=["mrcnn_class_logits", "mrcnn_bbox_fc",  "mrcnn_bbox", "mrcnn_mask"])

# iniciar entrenamiento del modelo
model.train(train_set, test_set, learning_rate=config.LEARNING_RATE, epochs=300, layers='all')



Starting at epoch 0. LR=0.001

Checkpoint Path: ./arandano_cfg_test_5_2_20220917T0844/mask_rcnn_arandano_cfg_test_5_2__{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_branch2a          (BatchNorm)
res3a_branch2b         (Conv2D)
bn3

/usr/local/lib/python3.8/dist-packages/keras/optimizers/optimizer_v2/gradient_descent.py:111: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/300


/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_2_grad/Reshape_1:0", shape=(6000,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_2_grad/Reshape:0", shape=(6000, 4), dtype=float32), dense_shape=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_2_grad/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_3_grad/Reshape_1:0", shape=(6000,), dtype=int32), values=Tensor("training/SGD/gradients/gradients/ROI/GatherV2_3_grad/Reshape:0", shape=(6000, 4), dtype=float32), dense_shape=Tensor("training/SGD/gradients/grad

146/146 [==============================] - 469s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 3.7550 - rpn_class_loss: 0.8334 - rpn_bbox_loss: 1.4259 - mrcnn_class_loss: 0.2708 - mrcnn_bbox_loss: 0.7239 - mrcnn_mask_loss: 0.5010 - val_loss: 2.8963 - val_rpn_class_loss: 0.6942 - val_rpn_bbox_loss: 0.8133 - val_mrcnn_class_loss: 0.4270 - val_mrcnn_bbox_loss: 0.4425 - val_mrcnn_mask_loss: 0.5193
Epoch 2/300
146/146 [==============================] - 360s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 1.9845 - rpn_class_loss: 0.3115 - rpn_bbox_loss: 0.4420 - mrcnn_class_loss: 0.3523 - mrcnn_bbox_loss: 0.4148 - mrcnn_mask_loss: 0.4638 - val_loss: 2.3844 - val_rpn_class_loss: 0.4212 - val_rpn_bbox_loss: 0.6239 - val_mrcnn_class_loss: 0.4572 - val_mrcnn_bbox_loss: 0.4057 - val_mrcnn_mask_loss: 0.4764
Epoch 3/300
146/146 [==============================] - 380s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 1.8846 - rpn_class_loss: 0.2854 - rpn_bbox_loss: 0.4209 - mrcnn_class_loss: 0.3777 -

...

Epoch 201/300
146/146 [==============================] - 345s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1712 - rpn_class_loss: 0.0013 - rpn_bbox_loss: 0.0161 - mrcnn_class_loss: 0.0335 - mrcnn_bbox_loss: 0.0181 - mrcnn_mask_loss: 0.1023 - val_loss: 6.7482 - val_rpn_class_loss: 3.8939 - val_rpn_bbox_loss: 1.0834 - val_mrcnn_class_loss: 1.0324 - val_mrcnn_bbox_loss: 0.2424 - val_mrcnn_mask_loss: 0.4961
Epoch 202/300
146/146 [==============================] - 357s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1664 - rpn_class_loss: 0.0012 - rpn_bbox_loss: 0.0139 - mrcnn_class_loss: 0.0328 - mrcnn_bbox_loss: 0.0176 - mrcnn_mask_loss: 0.1009 - val_loss: 6.7005 - val_rpn_class_loss: 3.9690 - val_rpn_bbox_loss: 1.0329 - val_mrcnn_class_loss: 0.9527 - val_mrcnn_bbox_loss: 0.2457 - val_mrcnn_mask_loss: 0.5003
Epoch 203/300
146/146 [==============================] - 398s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1680 - rpn_class_loss: 0.0011 - rpn_bbox_loss: 0.0152 - mrcnn_cl

Epoch 221/300
146/146 [==============================] - 344s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1537 - rpn_class_loss: 8.5472e-04 - rpn_bbox_loss: 0.0128 - mrcnn_class_loss: 0.0309 - mrcnn_bbox_loss: 0.0158 - mrcnn_mask_loss: 0.0934 - val_loss: 7.2737 - val_rpn_class_loss: 4.3862 - val_rpn_bbox_loss: 1.0332 - val_mrcnn_class_loss: 1.0776 - val_mrcnn_bbox_loss: 0.2430 - val_mrcnn_mask_loss: 0.5336
Epoch 222/300
146/146 [==============================] - 362s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1556 - rpn_class_loss: 0.0013 - rpn_bbox_loss: 0.0132 - mrcnn_class_loss: 0.0314 - mrcnn_bbox_loss: 0.0160 - mrcnn_mask_loss: 0.0938 - val_loss: 6.0162 - val_rpn_class_loss: 3.3774 - val_rpn_bbox_loss: 0.9266 - val_mrcnn_class_loss: 0.9317 - val_mrcnn_bbox_loss: 0.2670 - val_mrcnn_mask_loss: 0.5134
Epoch 223/300
146/146 [==============================] - 370s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1658 - rpn_class_loss: 0.0013 - rpn_bbox_loss: 0.0146 - mrcn

Epoch 241/300
146/146 [==============================] - 404s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1393 - rpn_class_loss: 7.9396e-04 - rpn_bbox_loss: 0.0107 - mrcnn_class_loss: 0.0283 - mrcnn_bbox_loss: 0.0135 - mrcnn_mask_loss: 0.0860 - val_loss: 7.0590 - val_rpn_class_loss: 4.1070 - val_rpn_bbox_loss: 0.9749 - val_mrcnn_class_loss: 1.1409 - val_mrcnn_bbox_loss: 0.2619 - val_mrcnn_mask_loss: 0.5743
Epoch 242/300
146/146 [==============================] - 362s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1441 - rpn_class_loss: 7.7766e-04 - rpn_bbox_loss: 0.0109 - mrcnn_class_loss: 0.0303 - mrcnn_bbox_loss: 0.0144 - mrcnn_mask_loss: 0.0876 - val_loss: 7.0388 - val_rpn_class_loss: 4.0977 - val_rpn_bbox_loss: 1.0047 - val_mrcnn_class_loss: 1.0942 - val_mrcnn_bbox_loss: 0.2656 - val_mrcnn_mask_loss: 0.5767
Epoch 243/300
146/146 [==============================] - 367s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1487 - rpn_class_loss: 8.7944e-04 - rpn_bbox_loss: 0.011

Epoch 261/300
146/146 [==============================] - 357s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1301 - rpn_class_loss: 8.5321e-04 - rpn_bbox_loss: 0.0100 - mrcnn_class_loss: 0.0271 - mrcnn_bbox_loss: 0.0124 - mrcnn_mask_loss: 0.0798 - val_loss: 7.0042 - val_rpn_class_loss: 4.2284 - val_rpn_bbox_loss: 0.9649 - val_mrcnn_class_loss: 1.0341 - val_mrcnn_bbox_loss: 0.2386 - val_mrcnn_mask_loss: 0.5381
Epoch 262/300
146/146 [==============================] - 384s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1326 - rpn_class_loss: 6.5643e-04 - rpn_bbox_loss: 0.0101 - mrcnn_class_loss: 0.0271 - mrcnn_bbox_loss: 0.0128 - mrcnn_mask_loss: 0.0819 - val_loss: 6.5442 - val_rpn_class_loss: 3.8739 - val_rpn_bbox_loss: 0.9285 - val_mrcnn_class_loss: 0.9224 - val_mrcnn_bbox_loss: 0.2490 - val_mrcnn_mask_loss: 0.5705
Epoch 263/300
146/146 [==============================] - 323s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1361 - rpn_class_loss: 7.9157e-04 - rpn_bbox_loss: 0.010

Epoch 281/300
146/146 [==============================] - 358s 2s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1260 - rpn_class_loss: 8.0492e-04 - rpn_bbox_loss: 0.0091 - mrcnn_class_loss: 0.0263 - mrcnn_bbox_loss: 0.0118 - mrcnn_mask_loss: 0.0780 - val_loss: 7.3202 - val_rpn_class_loss: 4.4341 - val_rpn_bbox_loss: 1.0301 - val_mrcnn_class_loss: 1.0551 - val_mrcnn_bbox_loss: 0.2352 - val_mrcnn_mask_loss: 0.5656
Epoch 282/300
146/146 [==============================] - 365s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1256 - rpn_class_loss: 7.6870e-04 - rpn_bbox_loss: 0.0095 - mrcnn_class_loss: 0.0256 - mrcnn_bbox_loss: 0.0120 - mrcnn_mask_loss: 0.0777 - val_loss: 7.3033 - val_rpn_class_loss: 4.3635 - val_rpn_bbox_loss: 0.9902 - val_mrcnn_class_loss: 1.0980 - val_mrcnn_bbox_loss: 0.2495 - val_mrcnn_mask_loss: 0.6021
Epoch 283/300
146/146 [==============================] - 390s 3s/step - batch: 72.5000 - size: 2.0000 - loss: 0.1244 - rpn_class_loss: 7.7499e-04 - rpn_bbox_loss: 0.009